In [128]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk import word_tokenize,sent_tokenize
import nltk

In [129]:
df=pd.read_csv('character-predictions.csv')
df[df['actual']!=df['isAlive']]

,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
846,847,0,0,0.047,0.953,Pate (Night's Watch),NaN,0,NaN,NaN,...,NaN,NaN,0,0,NaN,0,0,0,0.006689,1
848,849,1,1,0.968,0.032,Pate (Standfast),NaN,1,NaN,161.0,...,NaN,NaN,0,0,100.0,0,0,0,0.013378,0


In [130]:
identity=list()
none_index=list()
for i in range(df.shape[0]):
    response=requests.get("http://awoiaf.westeros.org/index.php/"+df.iloc[i]['name'].replace(' ','_'))
    results_page=BeautifulSoup(response.content,'lxml')
    try:
        identity.append(results_page.find('div',class_='mw-parser-output').find('p').get_text())
    except:
        identity.append(None)
        none_index.append(i)

In [131]:
len(identity)

1946

In [132]:
identity[363]='NaN'

In [133]:
identity_process=list()
for i in range(1946):
    x=nltk.pos_tag(word_tokenize(identity[i]))
    for j in range(len(x)):
        flag=0#find NN or NNP or not
        a=0#find is or was or not
        if x[j][0]=='is'or x[j][0]=='was':
            a=1#find is or was 
            for k in range(len(x)-j):
                if x[j+k][1]=='NNP'or x[j+k][1]=='NN':
                    identity_process.append(x[j+k][0])
                    flag=1 #find NNP or NN 
                    break
            if flag==0 and a==1:
                identity_process.append(0)
                break
            if flag==1 and a==1:
                break
    if a==0:
        identity_process.append(0)           
        
len(identity_process)

1946

In [134]:
kinship = ['descendant','son','mate','brother','daughter','wife','sister','child','cousin','uncle','mother','father','husband','wife','spouse','lover']

In [135]:
house=['Lannister','Tyrell','Arryn','Targaryen','Martell','Baratheon','Tully','Frey','Greyjoy','Stark','Bolton','Brave Companions','Brave','Blackwood',"Night's Watch",'Brave','Blackwood','Night','Ghiscari','Tyroshi']

In [136]:
df[np.array([identity_process[i] in house for i in range(1946)])].shape

(33, 33)

In [137]:
kinship_index=df[np.array([identity_process[i] in kinship for i in range(1946)])].index

In [138]:
member_index=df[np.array([identity_process[i]=='member' for i in range(1946)])].index

In [139]:
house_index=df[np.array([identity_process[i] in house for i in range(1946)])].index

In [140]:
member_index

Int64Index([   4,   15,   30,   47,   55,   91,   96,   99,  105,  108,
            ...
            1870, 1871, 1879, 1880, 1884, 1886, 1894, 1898, 1909, 1939],
           dtype='int64', length=265)

In [141]:
member_categories=[]
for i in member_index:
    name=df.iloc[i]['name']
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
    member_category=[]
    for li in lis:
        member_category.append(li.get_text())
    member_categories.append((i,member_category))

In [142]:
member_categories

[(4,
  ['Noblewomen',
   'Characters from Dorne',
   'House Santagar',
   'House Estermont',
   "Members of Arianne Martell's plot"]),
 (15,
  ['House Humble',
   'Characters from the Iron Islands',
   'Followers of Victarion Greyjoy',
   'Nobles',
   'Ship captains']),
 (30,
  ['House Waynwood',
   'Nobles',
   'Characters from the Vale',
   'Casualties of the First Blackfyre Rebellion']),
 (47,
  ['Characters from Westeros',
   'Members of the Kingswood Brotherhood',
   'Outlaws']),
 (55,
  ['House Targaryen',
   'Characters from the Crownlands',
   "Members of Aegon III Targaryen's court",
   "Members of Daeron I Targaryen's court",
   "Members of Baelor I Targaryen's court",
   "Members of Viserys II Targaryen's court",
   "Members of Aegon IV Targaryen's court",
   'Musicians',
   'Noblewomen',
   'Royal consorts',
   'Deaths by childbirth']),
 (91,
  ['House Rowan',
   'House Hightower',
   'Noblewomen',
   'Characters from the Reach']),
 (96, ['House Tyrell', 'Nobles', 'Characte

In [143]:
member_clean=dict()
for idx,i in enumerate(member_index):
    member_clean[i]=list()
    for j in member_categories[idx][1]:
        if (not('Casualties' in j) and  not('Characters' in j) and not('Dornishmen' in j) and not('Members' in j) and not('House' in j)
            and not('Supporters' in j) and not ('Followers' in j)):
            member_clean[i].append(j)

In [144]:
member_clean

{4: ['Noblewomen'],
 15: ['Nobles', 'Ship captains'],
 30: ['Nobles'],
 47: ['Outlaws'],
 55: ['Musicians', 'Noblewomen', 'Royal consorts', 'Deaths by childbirth'],
 91: ['Noblewomen'],
 96: ['Nobles'],
 99: ['Nobles', 'Pages'],
 105: ['Nobles', 'Squires'],
 108: ['Nobles', 'Second Sons', 'Mercenaries'],
 119: ['Nobles'],
 123: ['Nobles'],
 125: ['Noblewomen', 'Handmaids'],
 132: ['Bastards'],
 143: ['Nobles'],
 148: ['Nobles', 'Septons'],
 187: ['Nobles', 'Squires'],
 205: [],
 210: ['Nobles'],
 228: ['Nobles'],
 231: ['Kings of the Iron Islands', 'Monarchs', 'Nobles'],
 249: ['Nobles', 'Turncloaks in the War of the Five Kings'],
 257: ['Smallfolk', 'Sailors'],
 261: [],
 265: ['Nobles'],
 286: ['Claimants to the Iron Throne',
  'Dragonriders',
  'Nobles',
  'Princes of Dragonstone'],
 288: ['Nobles', 'Singers'],
 299: ['Nobles'],
 303: ['Noblewomen'],
 304: ['Greens', 'Noblewomen', 'Royal consorts'],
 306: ['Nobles', 'Ship captains'],
 309: ['Noblewomen'],
 330: ['Lords of Dragonston

In [145]:
for i in member_clean.keys():
        if len(member_clean[i])==1:
            identity_process[i]=member_clean[i][0]
        else:
            try:
                for j in member_clean[i]:
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0

In [146]:
identity_process

['Targaryen',
 'Lord',
 'knight',
 'consort',
 'Noblewomen',
 'Seven',
 'son',
 'Targaryen',
 'knight',
 'knight',
 'ranger',
 'boy',
 'squire',
 'farmer',
 'farmer',
 'Ship captains',
 'brother',
 'knight',
 'archmaester',
 'service',
 'knight',
 'knight',
 'sister',
 'knight',
 'peasant',
 'spearwife',
 'Night',
 'raider',
 'inhabitant',
 'guard',
 'Nobles',
 0,
 'nurse',
 'daughter',
 'mate',
 'son',
 'prostitute',
 'son',
 'service',
 'son',
 'maester',
 'builder',
 'Yandry',
 'archmaester',
 'whore',
 'Dothraki',
 'Ghiscari',
 'Outlaws',
 'son',
 'hand',
 'Seven',
 'knight',
 'brother',
 'Targaryen',
 'daughter',
 'Musicians',
 'mummer',
 'son',
 'Lord',
 'Lord',
 'son',
 'captain',
 'thief',
 0,
 'knight',
 'raider',
 'Night',
 'street',
 'Tyroshi',
 'raider',
 'focus',
 'inhabitant',
 'recruit',
 'brother',
 'head',
 'knight',
 'son',
 'raider',
 'slave',
 'supporter',
 'warrior',
 'ranger',
 'Lord',
 'Night',
 'son',
 'daughter',
 'daughter',
 'sister',
 'daughter',
 'daughter'

In [147]:
categories_kinship=dict()
for i in kinship_index:
    name=df.iloc[i]['name'].replace(' ','_')
#     print(name)
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
#     print(lis)
    category_kinship=[]
    for li in lis:
        if 'House' not in str(li) and 'Characters' not in str(li) and 'Members' not in str(li) and 'Casualties' not in str(li) and 'Dornishmen' not in str(li) and ('Supporters' not in str(li)) and ('Followers' not in str(li)):
            category_kinship.append(li.get_text())
    categories_kinship[i] = category_kinship

In [148]:
categories_kinship

{6: ['Hands of the King', 'Nobles', 'Princes of Dragonstone'],
 16: ['Smallfolk'],
 22: ['Smallfolk', 'Collaborators of the Brotherhood Without Banners'],
 33: ['Noblewomen'],
 34: ['Sailors'],
 35: ['Nobles'],
 37: ['Nobles'],
 39: ['Sailors'],
 48: ['Bastards', 'Squires'],
 52: ['Nobles', 'Castellans'],
 54: ['Noblewomen', 'Royal consorts'],
 57: ['Nobles'],
 60: ['Nobles'],
 73: [],
 76: ['Nobles'],
 84: ['Nobles', 'Princes of Dragonstone'],
 85: ['Blacks', 'Dragonriders', 'Noblewomen', 'Twins'],
 86: ['Noblewomen', 'Septas'],
 87: ['Dragonriders',
  'Noblewomen',
  'Royal consorts',
  "Veterans of Aegon's Conquest",
  'Warrior Women'],
 88: ['Noblewomen', 'Deaths by stabbing'],
 89: ['Noblewomen'],
 90: ['Blacks', 'Dragonriders', 'Noblewomen'],
 101: ['Lords of Winterfell',
  'Kings in the North',
  'Kings of the Trident and Kings of the Rivers and the Hills',
  'Monarchs',
  'Nobles',
  'Rebels',
  'Deaths by stabbing'],
 102: ['Smallfolk'],
 103: ['Nobles', 'Novices of the Citade

In [149]:
for i in categories_kinship.keys():
        if len(categories_kinship[i])==1:
            identity_process[i]=categories_kinship[i][0]
        else:
            try:
                for j in categories_kinship[i]:
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0

In [150]:
identity_process

['Targaryen',
 'Lord',
 'knight',
 'consort',
 'Noblewomen',
 'Seven',
 'Hands of the King',
 'Targaryen',
 'knight',
 'knight',
 'ranger',
 'boy',
 'squire',
 'farmer',
 'farmer',
 'Ship captains',
 'Smallfolk',
 'knight',
 'archmaester',
 'service',
 'knight',
 'knight',
 'Smallfolk',
 'knight',
 'peasant',
 'spearwife',
 'Night',
 'raider',
 'inhabitant',
 'guard',
 'Nobles',
 0,
 'nurse',
 'Noblewomen',
 'Sailors',
 'Nobles',
 'prostitute',
 'Nobles',
 'service',
 'Sailors',
 'maester',
 'builder',
 'Yandry',
 'archmaester',
 'whore',
 'Dothraki',
 'Ghiscari',
 'Outlaws',
 'Bastards',
 'hand',
 'Seven',
 'knight',
 'Castellans',
 'Targaryen',
 'Royal consorts',
 'Musicians',
 'mummer',
 'Nobles',
 'Lord',
 'Lord',
 'Nobles',
 'captain',
 'thief',
 0,
 'knight',
 'raider',
 'Night',
 'street',
 'Tyroshi',
 'raider',
 'focus',
 'inhabitant',
 'recruit',
 'brother',
 'head',
 'knight',
 'Nobles',
 'raider',
 'slave',
 'supporter',
 'warrior',
 'ranger',
 'Lord',
 'Night',
 'Princes of

In [152]:
categories_house=dict()
for i in house_index:
    name=df.iloc[i]['name'].replace(' ','_')
#     print(name)
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
#     print(lis)
    category_house=[]
    for li in lis:
        if 'House' not in str(li) and 'Characters' not in str(li) and 'Members' not in str(li) and 'Casualties' not in str(li) and 'Dornishmen' not in str(li) and ('Supporters' not in str(li)) and ('Followers' not in str(li)):
            category_house.append(li.get_text())
    categories_house[i] = category_house

In [153]:
for i in categories_house.keys():
        if len(categories_house[i])==1:
            identity_process[i]=categories_house[i][0]
        else:
            try:
                for j in categories_house[i]:
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0

In [165]:
df['identity']=identity_process

In [172]:
del df['S.No']

In [173]:
del df['actual']

In [174]:
del df['plod']

In [176]:
del df['pred']

In [178]:
del df['alive']

In [180]:
del df['title']

In [182]:
df.to_csv('data_being processing.csv')